In [4]:
#imports
import sklearn
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow.compat.v2.feature_column as fc

# Regressão linear com Tensor Flow - 2.x - Usando Pandas e Linear Regressor - Dados - Boston housing

In [ ]:
#load data
from sklearn.datasets import load_boston

boston_housing = load_boston()

In [ ]:
# boston_housing.data

In [ ]:
# print(boston_housing.DESCR)

In [6]:
# CONVERT DATA SET INTO DATAFRAME

df = pd.DataFrame(boston_housing.data, columns=boston_housing.feature_names)

In [ ]:
# df.head()

In [ ]:
#add the target column to the dataframe
df['MEDV'] = boston_housing.target 
#print the first 5 rows of the data frame
print(df.head())

In [8]:
#define the feature columns and target columns
features = df[boston_housing.feature_names]
target = 'MEDV'

In [ ]:
# df[boston_housing.feature_names]

In [9]:
#convert the feature columns into Tensorflow numeric column
feature_columns = [tf.feature_column.numeric_column(i) for i in features]

In [23]:
# define estimator linearRegressor

estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir='LinRegTrain')
clear_output()

In [24]:
# separando dados de treinamento e teste
from sklearn.model_selection import train_test_split
dftrain, dfeval = train_test_split(df, test_size=0.30, random_state=101)
print(dftrain.shape)
print(dfeval.shape)


(354, 14)
(152, 14)


In [25]:
# funções de input de dados

def make_input_fn(data_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    x = data_df[boston_housing.feature_names]
    y = data_df['MEDV']
    # batch_size=batch_size
    # num_epochs=num_epochs
    # shuffle=shuffle
   
    ds = tf.data.Dataset.from_tensor_slices((dict(x), y))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  
  return input_function

def make_input_fn_pred(data_df,num_epochs=1, shuffle=True, batch_size=32):
  def input_fn_pred():
    ds = tf.data.Dataset.from_tensor_slices(dict(data_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_fn_pred


In [35]:
# instanciando as funções de input de dados com os dataframes de treinamento e avaliação
train_input_fn = make_input_fn(dftrain,num_epochs=20,batch_size=3)
eval_input_fn = make_input_fn(dfeval, num_epochs=1, shuffle=False)


In [36]:

#train the model with 2000 steps
estimator.train(train_input_fn, steps=2000)
clear_output()


In [37]:
# avaliando o modelo
result = estimator.evaluate(eval_input_fn)

clear_output()
print(result)

{'average_loss': 42.953335, 'label/mean': 23.827631, 'loss': 42.76357, 'prediction/mean': 22.803696, 'global_step': 4016}


In [38]:

# Dados para realizar predição
df2 = pd.DataFrame(np.array([[0.00567, 16, 3.44,0,0.523,5.38,43,3.98,2,302,14.09,349.9,5.67], [0.00678, 12, 2.44,0,0.723,4.348,31,2.98,2,302,16.09,249.9,5.67]]),
                   columns=boston_housing.feature_names)
print(df2)
   

      CRIM    ZN  INDUS  CHAS    NOX  ...  RAD    TAX  PTRATIO      B  LSTAT
0  0.00567  16.0   3.44   0.0  0.523  ...  2.0  302.0    14.09  349.9   5.67
1  0.00678  12.0   2.44   0.0  0.723  ...  2.0  302.0    16.09  249.9   5.67

[2 rows x 13 columns]


In [43]:
# rodando predict
predict_input_fn = make_input_fn_pred(df2)
predictions = estimator.predict(input_fn=predict_input_fn)

In [44]:
list(predictions)



INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/canned/linear.py:1478: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  getter=tf.compat.v1.get_variable)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from LinRegTrain/model.ckpt-4016
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([17.856361], dtype=float32)},
 {'predictions': array([21.959055], dtype=float32)}]